In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
outputs = ['.\\Outputs', "scenario_comparison.gdb"]

if not os.path.exists(outputs[0]):
    os.makedirs(outputs[0])

gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

# Read in Boundaries

In [4]:
microzones =  r'E:\Tasks\Bike_Modeling_Data_Update\Processing\Inputs\Microzones_20240815.shp'
taz = ('taz', r'E:\Tasks\Bike_Modeling_Data_Update\Processing\Inputs\TAZ.gdb\TAZ_900', 'SA_TAZID')
city_area = ('cityarea',r'E:\Tasks\Bike_Modeling_Data_Update\Processing\Inputs\TAZ.gdb\CITYAREA_900', 'CityArea')
county = ('county', r'E:\Tasks\Bike_Modeling_Data_Update\Processing\Inputs\TAZ.gdb\COUNTY_900', 'CO_NAME')
subregion = ('subregion', r'E:\Tasks\Bike_Modeling_Data_Update\Processing\Inputs\SubCountyArea_2019.shp', 'NewSA')
centers = ('centers',r'E:\Data\Boundaries\WC2050Centers.shp', 'CENTER_ID')

In [5]:
# taz_df = pd.DataFrame.spatial.from_featureclass(taz[1])[[taz[2], 'SHAPE']]
# city_area_df = pd.DataFrame.spatial.from_featureclass(city_area[1])[[city_area[2], 'SHAPE']]
# county_df = pd.DataFrame.spatial.from_featureclass(county[1])[[county[2], 'SHAPE']]
# centers_df = pd.DataFrame.spatial.from_featureclass(centers[1])[[centers[2], 'SHAPE']]

In [6]:
# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(microzones)
# fieldmappings.addTable(centers)
# sj_centers = arcpy.SpatialJoin_analysis(microzones, centers, os.path.join(outputs[0], 'Microzones_20240815.shp'),'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, 'HAVE_THEIR_CENTER_IN')

# Store Scenarios

In [7]:
# scenarios
se2024 = ('se2024', r'E:\Projects\utah_bike_demand_model_2024')
se2024_bb = ('se2024_bb', r'E:\Projects\utah_bike_demand_model_2024_BB')
se2024_planned = ('se2024_planned', r'E:\Projects\utah_bike_demand_model_2024_Planned')
se2024_bb_planned = ('se2024_bb_planned', r'E:\Projects\utah_bike_demand_model_2024_BB_Planned')
se2050 = ('se2050', r'E:\Projects\utah_bike_demand_model_2050')
se2050_planned = ('se2050_planned', r'E:\Projects\utah_bike_demand_model_2050_Planned')
se2050_bb = ('se2050_bb', r'E:\Projects\utah_bike_demand_model_2050_BB')
se2050_bb_planned = ('se2050_bb_planned', r'E:\Projects\utah_bike_demand_model_2050_BB_Planned')

#  Calculate Scenario Metrics 

### Bike Volume

In [8]:
scenarios = [se2024, se2024_bb, se2024_planned, se2024_bb_planned, se2050, se2050_bb, se2050_planned, se2050_bb_planned]
geogs = [taz, city_area, county, subregion, centers]


for geog in geogs:
    print(f'working on {geog[0]}...')
    if geog == centers:
        geog_df = pd.DataFrame.spatial.from_featureclass(geog[1])[[geog[2], 'AreaName', 'SHAPE']]
    else:
        geog_df = pd.DataFrame.spatial.from_featureclass(geog[1])[[geog[2], 'SHAPE']]

    for scenario in scenarios:

        link_volume_pts = arcpy.FeatureToPoint_management(os.path.join(scenario[1], 
                                                                    f'Post_Process_Bike_Model_Outputs\Outputs\{scenario[0]}.gdb\links_bike_volume'), 
                                                                    os.path.join(gdb, f"{scenario[0]}_link_volume_pts"), 
                                                                    "INSIDE")

        target_features = geog[1]
        join_features = link_volume_pts
        output_features = os.path.join(gdb, f"{geog[0]}_bike_volume")

        fieldmappings = arcpy.FieldMappings()
        fieldmappings.addTable(geog[1])
        fieldmappings.addTable(link_volume_pts)

        fieldindex = fieldmappings.findFieldMapIndex('total_bvol')
        fieldmap = fieldmappings.getFieldMap(fieldindex)
        fieldmap.mergeRule = 'Sum'
        fieldmappings.replaceFieldMap(fieldindex, fieldmap)

        sj = arcpy.SpatialJoin_analysis(geog[1], link_volume_pts, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, 'INTERSECT')
        sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])[[geog[2], 'total_bvol']].copy()
        sj_df.columns = [geog[2], scenario[0]]
        sj_df[scenario[0]] = sj_df[scenario[0]].fillna(0).round(2).astype('float64')
        geog_df = geog_df.merge(sj_df, on=geog[2], how='left')
    
    geog_df.spatial.to_featureclass(location=os.path.join(gdb, f"{geog[0]}_bike_volume"), sanitize_columns=False) 


working on taz...


c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  

working on cityarea...
working on county...
working on subregion...
working on centers...


c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  

### Miles Travelled

In [33]:

scenarios = [se2024, se2024_bb, se2024_planned, se2024_bb_planned, se2050, se2050_bb, se2050_planned, se2050_bb_planned]
geogs = [taz, city_area, county, subregion, centers]


for geog in geogs:
    print(f'working on {geog[0]}...')
    if geog == centers:
        geog_df = pd.DataFrame.spatial.from_featureclass(geog[1])[[geog[2], 'AreaName', 'SHAPE']]
    else:
        geog_df = pd.DataFrame.spatial.from_featureclass(geog[1])[[geog[2], 'SHAPE']]

    for scenario in scenarios:

        link_volume_pts = arcpy.FeatureToPoint_management(os.path.join(scenario[1], 
                                                                    f'Post_Process_Bike_Model_Outputs\Outputs\{scenario[0]}.gdb\links_bike_volume'), 
                                                                    os.path.join(gdb, f"{scenario[0]}_link_volume_pts"), 
                                                                    "INSIDE")

        target_features = geog[1]
        join_features = link_volume_pts
        output_features = os.path.join(gdb, f"{geog[0]}_bike_volume")

        fieldmappings = arcpy.FieldMappings()
        fieldmappings.addTable(geog[1])
        fieldmappings.addTable(link_volume_pts)


        fieldindex = fieldmappings.findFieldMapIndex('miles_trv')
        fieldmap = fieldmappings.getFieldMap(fieldindex)
        fieldmap.mergeRule = 'Sum'
        fieldmappings.replaceFieldMap(fieldindex, fieldmap)

        sj = arcpy.SpatialJoin_analysis(geog[1], link_volume_pts, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, 'INTERSECT')
        sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])[[geog[2], 'miles_trv']].copy()
        sj_df.columns = [geog[2], scenario[0]]
        sj_df[scenario[0]] = sj_df[scenario[0]].fillna(0).round(2).astype('float64')
        geog_df = geog_df.merge(sj_df, on=geog[2], how='left')

        try:
            arcpy.management.Delete(link_volume_pts)
            arcpy.management.Delete(sj)
        except:
            print('failed to delete something... oh well...')
    
    geog_df.spatial.to_featureclass(location=os.path.join(gdb, f"{geog[0]}_miles_travelled"), sanitize_columns=False) 

working on taz...


c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  

working on cityarea...
working on county...
working on subregion...
working on centers...


c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  

### trip counts

In [34]:
scenarios = [se2024, se2024_bb, se2024_planned, se2024_bb_planned, se2050, se2050_bb, se2050_planned, se2050_bb_planned]
geogs = [taz, city_area, county, subregion, centers]
purposes = ['disc',  'mnt', 'mntnhb',  'recfam', 'reclng', 'recmtb', 'recoth', 'grade',  'univ', 'wrk', 'wrknhb', 'total']

for geog in geogs:
    print(f'working on {geog[0]}...')

    for purpose in purposes:
        # print(f'--working on {purpose}...')
        if geog == centers:
            geog_df = pd.DataFrame.spatial.from_featureclass(geog[1])[[geog[2], 'AreaName', 'SHAPE']]
        else:
            geog_df = pd.DataFrame.spatial.from_featureclass(geog[1])[[geog[2], 'SHAPE']]

        for scenario in scenarios:
            # print(f'----working on {scenario[0]}...')
            zone_trips_pts = arcpy.FeatureToPoint_management(os.path.join(scenario[1], 
                                                                        f'Post_Process_Bike_Model_Outputs\Outputs\{scenario[0]}.gdb\Microzone_Trip_Summaries'), 
                                                                        os.path.join(gdb, f"{scenario[0]}_zone_trips_pts"), 
                                                                        "INSIDE")

            target_features = geog[1]
            join_features = zone_trips_pts
            output_features = os.path.join(gdb, f"{geog[0]}_zone_trips")

            fieldmappings = arcpy.FieldMappings()
            fieldmappings.addTable(geog[1])
            fieldmappings.addTable(zone_trips_pts)

            fieldindex = fieldmappings.findFieldMapIndex(f'{purpose}_abk')
            fieldmap = fieldmappings.getFieldMap(fieldindex)
            fieldmap.mergeRule = 'Sum'
            fieldmappings.replaceFieldMap(fieldindex, fieldmap)

            fieldindex = fieldmappings.findFieldMapIndex(f'{purpose}_pbk')
            fieldmap = fieldmappings.getFieldMap(fieldindex)
            fieldmap.mergeRule = 'Sum'
            fieldmappings.replaceFieldMap(fieldindex, fieldmap)

            sj = arcpy.SpatialJoin_analysis(geog[1], zone_trips_pts, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", fieldmappings, 'INTERSECT')
            sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])[[geog[2], f'{purpose}_pbk', f'{purpose}_abk']].copy()
            sj_df.columns = [geog[2], f'{scenario[0]}_pbk', f'{scenario[0]}_abk']
            sj_df[f'{scenario[0]}_pbk'] = sj_df[f'{scenario[0]}_pbk'].fillna(0).round(2).astype('float64')
            sj_df[f'{scenario[0]}_abk'] = sj_df[f'{scenario[0]}_abk'].fillna(0).round(2).astype('float64')
            geog_df = geog_df.merge(sj_df, on=geog[2], how='left')

            try:
                arcpy.management.Delete(zone_trips_pts)
                arcpy.management.Delete(sj)
            except:
                print('failed to delete something... oh well...')
        
        geog_df.spatial.to_featureclass(location=os.path.join(gdb, f"{geog[0]}_{purpose}_trips"), sanitize_columns=False) 

working on taz...
working on cityarea...
working on county...
working on subregion...
working on centers...


In [ ]:
# trips produced by purpose, by taz

# trips attracted by purpose, by taz

# trips produced by purpose, by city area

# trips attracted by purpose, by city area

# trips attracted by purpose, in center


#     mmn_columns = ['Name', 'Oneway', 'Speed', 'AutoNetwork', 'BikeNetwork',
    #    'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'PedestrianTime',
    #    'Length_Miles', 'ConnectorNetwork', 'CartoCode', 'AADT', 'AADT_YR',
    #    'BIKE_L', 'BIKE_R', 'VERT_LEVEL']

        # for col in mmn_columns:
        #     fieldindex = fieldmappings.findFieldMapIndex(col)
        #     fieldmap = fieldmappings.getFieldMap(fieldindex)
        #     fieldmap.mergeRule = 'Sum'
        #     fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# Scenario Comparison